###The goal of this problem is to implement the "Median Maintenance" algorithm (covered in the Week 3 lecture on heap applications). The text file contains a list of the integers from 1 to 10000 in unsorted order; you should treat this as a stream of numbers, arriving one by one. Letting xi denote the ith number of the file, the kth median mk is defined as the median of the numbers x1,…,xk. (So, if k is odd, then mk is ((k+1)/2)th smallest number among x1,…,xk; if k is even, then mk is the (k/2)th smallest number among x1,…,xk.)

In the box below you should type the sum of these 10000 medians, modulo 10000 (i.e., only the last 4 digits). That is, you should compute (m1+m2+m3+⋯+m10000)mod10000. 

In [3]:
from heapq import heappush, heappop
import numpy as np

In [6]:
class simple_heap:
    def __init__(self):
        self.heap = []
        self.length = 0
        
    def get_child(self,h,parent):
        child1 = 2 * (parent + 1) - 1
        child2 = 2 * (parent + 1)
        
        if child1 < h.length - 1:
            if h.heap[child1] > h.heap[child2]:
                child = child2
            elif h.heap[child1] == h.heap[child2]:
                grandchild11 = 2 * (child1 + 1) - 1
                grandchild12 = 2 * (child1 + 1)
                grandchild21 = 2 * (child2 + 1) - 1
                grandchild22 = 2 * (child2 + 1)
                if grandchild11 > h.length - 1: 
                    # Scenario: Balanced heap for both children (no children exist for children nodes)
                    child = child2
                elif grandchild12 <= h.length - 1 and grandchild21 > h.length - 1:
                    # Scenario: Unbalanced heap (left child has children, while right child doesn't)
                    # thus, go to child with no children for simplicity
                    child = child2
                else: # The case that both children have two children each
                    final_descendant_left = grandchild11
                    final_descendant_left_right = grandchild12
                    
                    child = child1
                    # Check if left child is completely balanced
                    while final_descendant_left_right <= h.length - 1:
                        final_descendant_left = 2 * (final_descendant_left + 1) - 1
                        final_descendant_left_right = 2 * (final_descendant_left_right + 1)

                        if final_descendant_left_right > h.length - 1 and final_descendant_left > h.length - 1:
                            child = child2
            else:
                child = child1
                    
        elif child1 == h.length - 1:
            child = child1 # Child 2 is null
        else:
            child = h.length # out of range

        return child
        
        
    def pop(self):
        if self.length == 0:
            return None
        elif self.length == 1 or self.length == 2:
            self.length -= 1
            return self.heap.pop(0)
        elif self.length == 5: 
            # A strange case where removing the right node of the heap will make it imbalanced
            x = self.heap[0]
            self.heap[0] = self.heap[4]
            self.heap[4] = x
            self.heap.pop(4)
            self.length = 4

            if self.heap[0] > min(self.heap[1],self.heap[2]):
                if self.heap[1] < self.heap[2]: # Go to left child
                    temp = self.heap[0]
                    self.heap[0] = self.heap[1]
                    self.heap[1] = temp
                    if self.heap[1] > self.heap[3]:
                        temp = self.heap[1]
                        self.heap[1] = self.heap[3]
                        self.heap[3] = temp
                else: # Go to right child
                    temp = self.heap[0]
                    self.heap[0] = self.heap[2]
                    self.heap[2] = temp
            return x
        else:

            x = self.heap[0]
            self.heap[0] = self.heap[self.length - 1]
            self.heap[self.length - 1] = x
            self.heap.pop()
            self.length -= 1
            
            if self.length == 2:
                if self.heap[0] > self.heap[1]:
                    temp = self.heap[0]
                    self.heap[0] = self.heap[1]
                    self.heap[1] = temp
            else:
                parent = 0
                child = self.get_child(self,parent)
                
                while self.heap[child] < self.heap[parent]:
                    temp = self.heap[child]
                    self.heap[child] = self.heap[parent]
                    self.heap[parent] = temp
                    parent = child
                    child = self.get_child(self,parent)

                    if child > self.length - 1:
                        break
            return x
        
    def push(self,x):
        if self.length == 0:
            self.heap.append(x)
            self.length = 1
        elif self.length == 1:
            if self.heap[0] > x:
                self.heap.insert(0,x)
            else:
                self.heap.append(x)
            self.length = 2
        else:
            self.heap.append(x)
            self.length += 1
            child = self.length - 1
            parent = int((child+1)/2) - 1

            while self.heap[parent] > self.heap[child]:
                temp = self.heap[parent]
                self.heap[parent] = self.heap[child]
                self.heap[child] = temp

                if parent > 0:
                    child = parent
                    parent = int((parent+1)/2) - 1

In [7]:
x = simple_heap()
y = []
d = [-5, -3, -5, -1, -2, -4, -1]
for i in range(0,len(d)):
    x.push(d[i])
    heappush(y,d[i])
    
print("y heap before: ", y)
heappop(y)
print("y heap after: ", y)

print("x heap before", x.heap)
x.pop()
print("x heap after", x.heap)
print(" x == y: ", y == x.heap)

y heap before:  [-5, -3, -5, -1, -2, -4, -1]
y heap after:  [-5, -3, -4, -1, -2, -1]
x heap before [-5, -3, -5, -1, -2, -4, -1]
x heap after [-5, -3, -4, -1, -2, -1]
 x == y:  True


In [8]:
def Median_sum_modulo_efficient_tailored_test(data):
    heap_low = simple_heap()
    heap_high = simple_heap()
    m = []
    sum_m = 0
    len_data = 0
    for i in data:
        len_data += 1
        if len_data == 1:
            print("Heap_low before push (len = 1): ", heap_low.heap, ", i = ", i)
            heap_low.push(-i)
            print("Heap_low after push: ", heap_low.heap)
        else:
            if  i <= -heap_low.heap[0]:
                print("Heap_low before push (i < h[0]): ", heap_low.heap, ", i = ", i)
                heap_low.push(-i)
                print("Heap_low after push: ", heap_low.heap)
            else:
                print("Heap_high before push: ", heap_high.heap, ", i = ", i)
                heap_high.push(i)
                print("Heap_high after push: ", heap_high.heap)
            # Next two if statement ensure that the heaps are balanced with low heap at most one extra
            # item more than the high heap. No need to use loops here since we are iterating one datum
            # at a time
            if heap_low.length < heap_high.length:
                print("Heap_low before push (heap_high.pop): ", heap_low.heap, ", heap_high", heap_high.heap)
                heap_low.push(-heap_high.pop())
                print("Heap_low after push (heap_high.pop): ", heap_low.heap, ", heap_high", heap_high.heap)

            elif heap_low.length > heap_high.length + 1: # We want low heap to be at most one item more than high heap
                print("Heap_high before push (heap_low.pop): ", heap_high.heap, ", heap_low", heap_low.heap)
                heap_high.push(-heap_low.pop())
                print("Heap_high before push (heap_low.pop): ", heap_high.heap, ", heap_low", heap_low.heap)
        
        median = -heap_low.heap[0]
        
        m.append(median)
        sum_m += median
    print("Heap_low = ", heap_low.heap)
    print("Heap_high = ", heap_high.heap)
    print("Median_Tailored = ", m)
    return sum_m%len_data

In [9]:
def Median_sum_modulo_efficient_test(data):
    
    heap_low = []
    heap_high = []
    m = []
    sum_m = 0
    len_data = 0
    len_heap_low = 0
    len_heap_high = 0
    for i in data:
        len_data += 1
        if len_data == 1:
            print("Heap_low before push (len = 1): ", heap_low, ", i = ", i)
            heappush(heap_low,-i)
            len_heap_low += 1
            print("Heap_low after push: ", heap_low)
            
        else:
            if  i <= -heap_low[0]:
                print("Heap_low before push (i < h[0]): ", heap_low, ", i = ", i)
                heappush(heap_low,-i)
                len_heap_low += 1
                print("Heap_low after push: ", heap_low)

            else:
                print("Heap_high before push: ", heap_high, ", i = ", i)
                heappush(heap_high,i)
                len_heap_high += 1
                print("Heap_high after push: ", heap_high)

            # Next two if statement ensure that the heaps are balanced with low heap at most one extra
            # item more than the high heap. No need to use loops here since we are iterating one datum
            # at a time
            if len_heap_low < len_heap_high:
                print("Heap_low before push (heap_high.pop): ", heap_low, ", heap_high", heap_high)
                heappush(heap_low,-heappop(heap_high))
                len_heap_low += 1
                len_heap_high -= 1
                print("Heap_low after push (heap_high.pop): ", heap_low, ", heap_high", heap_high)
                

            elif len_heap_low > len_heap_high +1: # We want low heap to be at most one item more than high heap
                print("Heap_high before push (heap_low.pop): ", heap_high, ", heap_low", heap_low)
                heappush(heap_high,-heappop(heap_low))
                len_heap_low -= 1
                len_heap_high += 1
                print("Heap_high after push (heap_low.pop): ", heap_high, ", heap_low", heap_low)
        
        median = -heap_low[0]
        
        m.append(median)
        sum_m += median
    print("Heap_low = ", heap_low)
    print("Heap_high = ", heap_high)
    print("Median_Original = ", m)
    return sum_m%len_data

In [10]:
def Median_sum_modulo_efficient_tailored(data):
    heap_low = simple_heap()
    heap_high = simple_heap()
    m = []
    sum_m = 0
    len_data = 0
    for i in data:
        len_data += 1
        if len_data == 1:
            heap_low.push(-i)
        else:
            if  i <= -heap_low.heap[0]:
                heap_low.push(-i)
            else:
                heap_high.push(i)
            # Next two if statement ensure that the heaps are balanced with low heap at most one extra
            # item more than the high heap. No need to use loops here since we are iterating one datum
            # at a time
            if heap_low.length < heap_high.length:
                heap_low.push(-heap_high.pop())

            elif heap_low.length > heap_high.length + 1: # We want low heap to be at most one item more than high heap
                heap_high.push(-heap_low.pop())
        
        median = -heap_low.heap[0]
        
        m.append(median)
        sum_m += median

    return sum_m%len_data

In [11]:
def Median_sum_modulo_efficient(data):
    
    heap_low = []
    heap_high = []
    m = []
    sum_m = 0
    len_data = 0
    len_heap_low = 0
    len_heap_high = 0
    for i in data:
        len_data += 1
        if len_data == 1:
            heappush(heap_low,-i)
            len_heap_low += 1
            
        else:
            if  i <= -heap_low[0]:
                heappush(heap_low,-i)
                len_heap_low += 1

            else:
                heappush(heap_high,i)
                len_heap_high += 1

            # Next two if statement ensure that the heaps are balanced with low heap at most one extra
            # item more than the high heap. No need to use loops here since we are iterating one datum
            # at a time
            if len_heap_low < len_heap_high:
                heappush(heap_low,-heappop(heap_high))
                len_heap_low += 1
                len_heap_high -= 1
                

            elif len_heap_low > len_heap_high +1: # We want low heap to be at most one item more than high heap
                heappush(heap_high,-heappop(heap_low))
                len_heap_low -= 1
                len_heap_high += 1
        
        median = -heap_low[0]
        
        m.append(median)
        sum_m += median

    return sum_m%len_data

In [13]:
d = []
counter = 0
for i in range(0,10000):
    data = np.random.randint(10,size=50)
    if Median_sum_modulo_efficient_tailored(data) != Median_sum_modulo_efficient(data):
        d = data
        counter += 1
print("counter = ", counter)
print("d = ", d)
d

counter =  0
d =  []


[]

In [14]:
file = open("Median.txt")
lines = file.read().splitlines()
file.close()
data = list(map(lambda x: int(x), lines))
#data = d
#data

In [15]:
print("Tailored: ", Median_sum_modulo_efficient_tailored(data))

Tailored:  1213


In [16]:
print("Original: ", Median_sum_modulo_efficient(data))

Original:  1213
